In [218]:
"""
TAT (Turnaround Time) Calculator System
=====================================

A comprehensive system for calculating adjusted timestamps for Purchase Order (PO) 
processing workflow stages using a configurable, dependency-driven approach.

This implementation replaces complex Excel formulas with maintainable Python code
that supports dynamic expressions, stage dependencies, and transparent reasoning.
"""

import json
import ast
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple, Union, Any
from pathlib import Path
import pandas as pd
from pydantic import BaseModel, Field, validator
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class ProcessFlow(BaseModel):
    """Process flow metadata for a stage"""
    critical_path: bool
    parallel_processes: List[str] = Field(default_factory=list)
    handoff_points: List[str] = Field(default_factory=list)
    process_type: str
    team_owner: str


class FallbackCalculation(BaseModel):
    """Fallback calculation configuration"""
    expression: str
    
    @validator('expression')
    def validate_expression(cls, v):
        """Validate that expression is not empty"""
        if not v.strip():
            raise ValueError("Expression cannot be empty")
        return v


class StageConfig(BaseModel):
    """Configuration for a single stage"""
    name: str
    actual_timestamp: Optional[str] = None
    preceding_stage: Optional[Union[str, List[str]]] = None
    process_flow: ProcessFlow
    fallback_calculation: FallbackCalculation
    lead_time: int = Field(ge=0, description="Lead time in days")
    
    @validator('preceding_stage')
    def validate_preceding_stage(cls, v):
        """Convert single string to list for consistency"""
        if isinstance(v, str):
            return [v]
        return v


class StagesConfig(BaseModel):
    """Complete stages configuration"""
    stages: Dict[str, StageConfig]
    
    @validator('stages')
    def validate_stage_ids(cls, v):
        """Ensure all stage IDs are valid"""
        for stage_id in v.keys():
            if not stage_id.strip():
                raise ValueError("Stage ID cannot be empty")
        return v


class TATCalculator:
    """
    Core TAT calculation engine that processes PO data through configurable stages.
    
    Features:
    - Configuration-driven stage definitions
    - Dynamic expression evaluation with custom functions
    - Dependency resolution with cycle detection
    - Memoization for performance optimization
    - Comprehensive audit trails and reasoning
    """
    
    def __init__(self, config_path: str = "stages_config.json"):
        """
        Initialize the TAT Calculator
        
        Args:
            config_path: Path to the stages configuration JSON file
        """
        self.config = self._load_config(config_path)
        self._validate_config()
        self.calculated_adjustments: Dict[str, Tuple[Optional[datetime], Dict[str, Any]]] = {}
        
    def _load_config(self, config_path: str) -> StagesConfig:
        """Load and validate configuration from JSON file"""
        try:
            with open(config_path, 'r') as f:
                config_data = json.load(f)
            return StagesConfig(**config_data)
        except FileNotFoundError:
            logger.error(f"Configuration file not found: {config_path}")
            raise
        except json.JSONDecodeError as e:
            logger.error(f"Invalid JSON in configuration file: {e}")
            raise
        except Exception as e:
            logger.error(f"Error loading configuration: {e}")
            raise
    
    def _validate_config(self):
        """Validate configuration for circular dependencies"""
        # Build dependency graph
        graph = {}
        for stage_id, stage in self.config.stages.items():
            graph[stage_id] = stage.preceding_stage or []
        
        # Check for cycles using DFS
        visited = set()
        rec_stack = set()
        
        def has_cycle(node):
            if node in rec_stack:
                return True
            if node in visited:
                return False
                
            visited.add(node)
            rec_stack.add(node)
            
            for neighbor in graph.get(node, []):
                if neighbor in graph and has_cycle(neighbor):
                    return True
            
            rec_stack.remove(node)
            return False
        
        for stage_id in graph:
            if stage_id not in visited:
                if has_cycle(stage_id):
                    raise ValueError(f"Circular dependency detected involving stage {stage_id}")
    
    def _get_date_value(self, field_name: str, po_row: pd.Series) -> Optional[datetime]:
        """
        Extract datetime value from PO data with robust handling
        
        Args:
            field_name: Name of the field to extract
            po_row: Pandas Series containing PO data
            
        Returns:
            datetime object or None if not available/valid
        """
        if field_name not in po_row.index:
            logger.warning(f"Field '{field_name}' not found in PO data")
            return None
            
        value = po_row[field_name]
        
        # Handle various data types
        if pd.isna(value) or value == "" or value == "NA":
            return None
            
        if isinstance(value, datetime):
            return value
            
        if isinstance(value, str):
            try:
                # Try common datetime formats
                for fmt in ["%Y-%m-%d", "%Y-%m-%d %H:%M:%S", "%m/%d/%Y", "%d/%m/%Y"]:
                    try:
                        return datetime.strptime(value, fmt)
                    except ValueError:
                        continue
                        
                # Try pandas datetime parsing as fallback
                return pd.to_datetime(value)
            except:
                logger.warning(f"Could not parse date from field '{field_name}': {value}")
                return None
                
        return None
    
    def _evaluate_expression(self, expression: str, po_row: pd.Series) -> Tuple[Optional[datetime], str]:
        try:
            tree = ast.parse(expression, mode='eval')
            result = self._eval_node(tree.body, po_row)
            
            if isinstance(result, datetime):
                return result, f"Calculation: {expression} = {result.strftime('%Y-%m-%d %H:%M:%S')}"
            elif result is not None:
                # Handle non-datetime results (for testing)
                return None, f"Calculation returned non-datetime: {expression} = {result}"
            else:
                return None, f"Calculation failed: {expression}"
                
        except Exception as e:
            logger.error(f"Error evaluating expression '{expression}': {e}")
            return None, f"Calculation error: {expression} ({str(e)})"
    

    def _eval_node(self, node: ast.AST, po_row: pd.Series) -> Any:
        """Recursively evaluate AST nodes"""
        if isinstance(node, ast.Name):
            return po_row.get(node.id)
        
        elif isinstance(node, ast.Constant):
            return node.value
        
        elif isinstance(node, ast.List):
            # Handle list literals like ['5'], ['2']
            return [self._eval_node(elt, po_row) for elt in node.elts]
        
        elif isinstance(node, ast.Compare):
            # Handle comparisons like pi_applicable==1
            left = self._eval_node(node.left, po_row)
            op = node.ops[0]
            right = self._eval_node(node.comparators[0], po_row)
            
            if isinstance(op, ast.Eq):
                return left == right
            elif isinstance(op, ast.NotEq):
                return left != right
            elif isinstance(op, ast.Lt):
                return left < right
            elif isinstance(op, ast.LtE):
                return left <= right
            elif isinstance(op, ast.Gt):
                return left > right
            elif isinstance(op, ast.GtE):
                return left >= right
            else:
                raise ValueError(f"Unsupported comparison operator: {type(op).__name__}")
        
        elif isinstance(node, ast.Call):
            if isinstance(node.func, ast.Name):
                func_name = node.func.id
                
                # Handle iff with lazy evaluation
                if func_name == 'iff':
                    if len(node.args) == 3:
                        condition = self._eval_node(node.args[0], po_row)
                        if condition:
                            return self._eval_node(node.args[1], po_row)
                        else:
                            return self._eval_node(node.args[2], po_row)
                    raise ValueError("iff requires exactly 3 arguments")
                
                # For other functions, evaluate arguments first
                args = [self._eval_node(arg, po_row) for arg in node.args]
                
                if func_name == 'max':
                    valid_dates = [arg for arg in args if isinstance(arg, datetime)]
                    return max(valid_dates) if valid_dates else None
                
                elif func_name == 'add_days':
                    if len(args) >= 2 and isinstance(args[0], datetime) and isinstance(args[1], (int, float)):
                        return args[0] + timedelta(days=int(args[1]))
                    return None
                
                else:
                    raise ValueError(f"Unknown function: {func_name}")
        
        else:
            raise ValueError(f"Unsupported AST node type: {type(node).__name__}")

    def calculate_adjusted_timestamp(self, stage_id: str, po_row: pd.Series) -> Tuple[Optional[datetime], Dict[str, Any]]:
        """
        Calculate adjusted timestamp for a specific stage using priority logic:
        
        1. Precedence-based calculation (from dependencies + lead time)
        2. Actual timestamp comparison (if available)
        3. Fallback calculation (last resort)
        
        Args:
            stage_id: ID of the stage to calculate
            po_row: PO data row
            
        Returns:
            Tuple of (calculated_timestamp, calculation_details)
        """
        # Check if already calculated (memoization)
        if stage_id in self.calculated_adjustments:
            return self.calculated_adjustments[stage_id]
        
        if stage_id not in self.config.stages:
            logger.error(f"Stage {stage_id} not found in configuration")
            return None, {"method": "error", "reason": f"Stage {stage_id} not found"}
        
        stage = self.config.stages[stage_id]
        
        # Initialize calculation details
        calc_details = {
            "method": None,
            "source": None,
            "base_date": None,
            "lead_time_applied": stage.lead_time,
            "decision_reason": None,
            "dependencies": [],
            "actual_field": None,
            "actual_value": None,
            "precedence_value": None,
            "final_choice": None
        }
        
        # 1. Calculate precedence-based timestamp
        precedence_timestamp = None
        if stage.preceding_stage:
            dependencies = []
            preceding_timestamps = []
            
            # Check if it's a single conditional expression
            if len(stage.preceding_stage) == 1 and isinstance(stage.preceding_stage[0], str) and 'iff(' in stage.preceding_stage[0]:
                # Evaluate the conditional to get the list of stage IDs
                try:
                    result = self._eval_node(ast.parse(stage.preceding_stage[0], mode='eval').body, po_row)
                    preceding_stage_ids = result if isinstance(result, list) else [result] if result else []
                except Exception as e:
                    logger.error(f"Error evaluating conditional preceding stage: {e}")
                    preceding_stage_ids = []
            else:
                preceding_stage_ids = stage.preceding_stage
            
            # Process the stage IDs
            for prec_stage_id in preceding_stage_ids:
                prec_stage_id = str(prec_stage_id)  # Ensure it's a string
                if prec_stage_id in self.config.stages:
                    prec_timestamp, prec_details = self.calculate_adjusted_timestamp(prec_stage_id, po_row)
                    if prec_timestamp:
                        preceding_timestamps.append(prec_timestamp)
                        dependencies.append({
                            "stage_id": prec_stage_id,
                            "stage_name": self.config.stages[prec_stage_id].name,
                            "timestamp": prec_timestamp.isoformat(),
                            "method": prec_details["method"] if isinstance(prec_details, dict) else "legacy"
                        })
            
            calc_details["dependencies"] = dependencies
            if preceding_timestamps:
                base_timestamp = max(preceding_timestamps)
                precedence_timestamp = base_timestamp + timedelta(days=stage.lead_time)
                calc_details["precedence_value"] = precedence_timestamp.isoformat()
                calc_details["base_date"] = base_timestamp.isoformat()
        
        # 2. Extract and get actual timestamp (evaluate as expression or field)
        actual_timestamp = None
        actual_formula = None
        if stage.actual_timestamp:
            actual_timestamp, actual_formula = self._evaluate_expression(stage.actual_timestamp, po_row)
            calc_details["actual_field"] = stage.actual_timestamp
            if actual_timestamp:
                calc_details["actual_value"] = actual_timestamp.isoformat()
        
        # 3. Determine final timestamp and method
        final_timestamp = None
        if precedence_timestamp and actual_timestamp:
            if actual_timestamp >= precedence_timestamp:
                final_timestamp = actual_timestamp
                calc_details["method"] = "actual_over_precedence"
                calc_details["source"] = actual_formula
                calc_details["decision_reason"] = f"Actual date ({actual_timestamp.strftime('%Y-%m-%d')}) is later than precedence date ({precedence_timestamp.strftime('%Y-%m-%d')})"
                calc_details["final_choice"] = "actual"
            else:
                final_timestamp = precedence_timestamp
                calc_details["method"] = "precedence_over_actual"
                calc_details["source"] = f"Calculated from dependencies"
                calc_details["decision_reason"] = f"Precedence stage's timestamp ({precedence_timestamp.strftime('%Y-%m-%d')}) is later than actual ({actual_timestamp.strftime('%Y-%m-%d')})"
                calc_details["final_choice"] = "precedence"
        
        elif actual_timestamp:
            final_timestamp = actual_timestamp
            calc_details["method"] = "actual_only"
            calc_details["source"] = actual_formula
            calc_details["decision_reason"] = "Using actual timestamp"
            calc_details["final_choice"] = "actual"
        
        elif precedence_timestamp:
            final_timestamp = precedence_timestamp
            calc_details["method"] = "precedence_only"
            calc_details["source"] = f"Calculated from dependencies + {stage.lead_time} days"
            calc_details["decision_reason"] = "No actual timestamp available, using precedence calculation"
            calc_details["final_choice"] = "precedence"
        
        # 4. Fallback calculation if no valid timestamp
        if not final_timestamp:
            fallback_result, fallback_formula = self._evaluate_expression(
                stage.fallback_calculation.expression, po_row
            )
            if fallback_result:
                final_timestamp = fallback_result + timedelta(days=stage.lead_time)
                calc_details["method"] = "fallback"
                calc_details["source"] = stage.fallback_calculation.expression
                calc_details["base_date"] = fallback_result.isoformat()
                calc_details["decision_reason"] = "No precedence or actual data available, using fallback expression"
                calc_details["final_choice"] = "fallback"
            else:
                calc_details["method"] = "failed"
                calc_details["decision_reason"] = "No valid calculation method available"
        
        # Cache result
        result = (final_timestamp, calc_details)
        self.calculated_adjustments[stage_id] = result
        
        return result
    def _extract_actual_field(self, expression: str) -> Optional[str]:
        """Extract the 'actual' field from a max() expression"""
        # This method should be at the same level as _eval_node, not inside it
        try:
            tree = ast.parse(expression, mode='eval')
            if isinstance(tree.body, ast.Call) and isinstance(tree.body.func, ast.Name):
                if tree.body.func.id == 'max' and tree.body.args:
                    first_arg = tree.body.args[0]
                    if isinstance(first_arg, ast.Name):
                        return first_arg.id
        except:
            pass
        return None
        

    
    def calculate_tat(self, po_row: pd.Series) -> Dict[str, Any]:
        """
        Calculate TAT for all stages of a PO
        
        Args:
            po_row: Pandas Series containing PO data
            
        Returns:
            Dictionary with complete TAT calculation results
        """
        # Clear cache for new calculation
        self.calculated_adjustments = {}
        
        result = {
            "po_id": po_row.get('po_razin_id', 'Unknown'),
            "calculation_date": datetime.now().isoformat(),
            "summary": {
                "total_stages": len(self.config.stages),
                "calculated_stages": 0,
                "methods_used": {
                    "actual_only": 0,
                    "precedence_only": 0,
                    "actual_over_precedence": 0,
                    "precedence_over_actual": 0,
                    "fallback": 0,
                    "failed": 0
                }
            },
            "stages": {}
        }
        
        # Calculate each stage
        for stage_id, stage_config in self.config.stages.items():
            timestamp, calc_details = self.calculate_adjusted_timestamp(stage_id, po_row)
            
            # Update summary statistics
            if timestamp:
                result["summary"]["calculated_stages"] += 1
            
            method = calc_details.get("method", "unknown") if isinstance(calc_details, dict) else "legacy"
            if method in result["summary"]["methods_used"]:
                result["summary"]["methods_used"][method] += 1
            
            # Create clean stage result
            stage_result = {
                "name": stage_config.name,
                "timestamp": timestamp.isoformat() if timestamp else None,
                "calculation": self._format_calculation_summary(calc_details, stage_config),
                "process_flow": {
                    "team_owner": stage_config.process_flow.team_owner,
                    "process_type": stage_config.process_flow.process_type,
                    "critical_path": stage_config.process_flow.critical_path,
                    "handoff_points": stage_config.process_flow.handoff_points
                },
                "dependencies": calc_details.get("dependencies", []) if isinstance(calc_details, dict) else []
            }
            
            result["stages"][stage_id] = stage_result
        
        # Calculate completion rate
        result["summary"]["completion_rate"] = round(
            result["summary"]["calculated_stages"] / result["summary"]["total_stages"] * 100, 2
        ) if result["summary"]["total_stages"] > 0 else 0
        
        return result
    
    def _format_calculation_summary(self, calc_details: Dict[str, Any], stage_config: StageConfig) -> Dict[str, Any]:
        """
        Format calculation details into a clean, readable summary
        
        Args:
            calc_details: Raw calculation details
            stage_config: Stage configuration
            
        Returns:
            Clean calculation summary
        """
        if not isinstance(calc_details, dict):
            return {"method": "legacy", "summary": str(calc_details)}
        
        method = calc_details.get("method", "unknown")
        
        summary = {
            "method": method,
            "source": calc_details.get("source"),
            "decision": calc_details.get("decision_reason"),
            "lead_time_days": calc_details.get("lead_time_applied", 0)
        }
        
        # Add method-specific details
        if method == "actual_over_precedence":
            summary.update({
                "actual_date": calc_details.get("actual_value"),
                "precedence_date": calc_details.get("precedence_value"),
                "reason": "Actual timestamp is later than calculated precedence"
            })
        elif method == "precedence_over_actual":
            summary.update({
                "actual_date": calc_details.get("actual_value"),
                "precedence_date": calc_details.get("precedence_value"), 
                "reason": "Calculated precedence is later than actual timestamp"
            })
        elif method == "precedence_only":
            summary.update({
                "base_date": calc_details.get("base_date"),
                "reason": "No actual timestamp available"
            })
        elif method == "actual_only":
            summary.update({
                "actual_field": calc_details.get("actual_field"),
                "reason": "No dependency chain available"
            })
        elif method == "fallback":
            summary.update({
                "base_date": calc_details.get("base_date"),
                "expression": calc_details.get("source"),
                "reason": "Using fallback calculation"
            })
        
        return summary
    
    def process_batch(self, df: pd.DataFrame) -> List[Dict[str, Any]]:
        """
        Process multiple POs in batch
        
        Args:
            df: DataFrame containing multiple PO rows
            
        Returns:
            List of TAT calculation results
        """
        results = []
        
        for index, row in df.iterrows():
            try:
                result = self.calculate_tat(row)
                results.append(result)
                logger.info(f"Processed PO: {result['po_id']}")
            except Exception as e:
                logger.error(f"Error processing row {index}: {e}")
                results.append({
                    "po_id": row.get('po_razin_id', f'Row_{index}'),
                    "error": str(e),
                    "calculation_date": datetime.now().isoformat()
                })
        
        return results

    def export_to_excel(self, df: pd.DataFrame, results: List[Dict[str, Any]], output_file: str):
        """
        Export original data + calculated timestamps to Excel
        
        Args:
            df: Original DataFrame
            results: TAT calculation results
            output_file: Output Excel file path
        """
        # Create a copy of the original dataframe
        export_df = df.copy()
        
        # Add calculated timestamps for each stage
        for result in results:
            if 'stages' not in result:
                continue
                
            po_id = result['po_id']
            po_index = export_df[export_df['po_razin_id'] == po_id].index
            
            if len(po_index) > 0:
                idx = po_index[0]
                
                # Add calculated timestamps
                for stage_id, stage_data in result['stages'].items():
                    # Use stage name instead of ID for column name
                    stage_name = stage_data['name']
                    col_name = f"{stage_name}_Date"
                    timestamp = stage_data['timestamp']
                    # Convert timestamp to date only
                    if timestamp:
                        date = pd.to_datetime(timestamp).date()
                        export_df.loc[idx, col_name] = date
                    else:
                        export_df.loc[idx, col_name] = None
        
        # Save to Excel
        export_df.to_excel(output_file, index=False)
        logger.info(f"Results exported to: {output_file}")


if __name__ == "__main__":
    print("TAT Calculator System - Enhanced with Excel Export")
    print("Usage:")
    print("1. calculator = TATCalculator()")
    print("2. results = calculator.process_batch(df)")
    print("3. calculator.export_to_excel(df, results, 'output.xlsx')")


TAT Calculator System - Enhanced with Excel Export
Usage:
1. calculator = TATCalculator()
2. results = calculator.process_batch(df)
3. calculator.export_to_excel(df, results, 'output.xlsx')


/var/folders/cq/q5x0b26j433crqf4l5qsxqc00000gn/T/ipykernel_18389/3236826720.py:40: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('expression')
/var/folders/cq/q5x0b26j433crqf4l5qsxqc00000gn/T/ipykernel_18389/3236826720.py:57: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('preceding_stage')
/var/folders/cq/q5x0b26j433crqf4l5qsxqc00000gn/T/ipykernel_18389/3236826720.py:69: PydanticDeprecatedSince20: Pydantic 

In [ ]:
"""
Complete TAT Calculation Runner
==============================

This script runs the complete TAT calculation system on your Excel data
and generates comprehensive reports and analytics.
"""

import pandas as pd
import json
import logging
from datetime import datetime, timedelta
from pathlib import Path
import sys
import traceback

# Set up comprehensive logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('tat_calculation.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)


class TATRunner:
    """Complete TAT calculation runner with enhanced reporting"""
    
    def __init__(self, excel_file: str = "ts_big.xlsx", config_file: str = "stages_config.json"):
        self.excel_file = excel_file
        self.config_file = config_file
        self.df = None
        self.calculator = None
        self.results = []
        
    def setup(self):
        """Set up the calculator and load data"""
        logger.info("Setting up TAT calculation environment...")
        
        # Load Excel data
        self.load_excel_data()
        
        # Initialize calculator
        self.initialize_calculator()
        
        logger.info("Setup completed successfully!")
        
    def load_excel_data(self):
        """Load and prepare Excel data"""
        logger.info(f"Loading Excel file: {self.excel_file}")
        
        try:
            self.df = pd.read_excel(self.excel_file)
            logger.info(f"Loaded {len(self.df)} rows and {len(self.df.columns)} columns")
            
            # Clean column names
            self.df.columns = self.df.columns.str.strip()
            
            # Convert date columns
            self.convert_date_columns()
            
            # Validate required columns
            self.validate_required_columns()
            
        except Exception as e:
            logger.error(f"Error loading Excel file: {e}")
            raise
            
    def convert_date_columns(self):
        """Convert relevant columns to datetime"""
        date_columns = [
            'po_created_date', 'po_approval_date', 'supplier_confirmation_date',
            'pi_invoice_approval_date', 'pi_payment_date', 'receive_first_prd_date',
            'prd_reconfirmed_date', 'po_im_date_value', 'po_sm_date_value',
            'batch_created_ts', 'sm_signoff_ts', 'ci_invoice_approval_date',
            'ci_payment_date', 'qc_schedule_date', 'ffw_booking_ts', 'spd_ts',
            'stock_pickup_date', 'shipment_creation_date', 'shipment_in_transit_date',
            'bi_invoice_approval_date', 'bi_payment_date', 'ffwp_telex_release_date',
            'shipment_stock_delivery_date', 'item_receipt_date', 'actual_cargo_pick_up_date',
            'actual_shipping_date', 'actual_arrival_date', 'actual_delivery_date'
        ]
        
        for col in date_columns:
            if col in self.df.columns:
                original_type = self.df[col].dtype
                self.df[col] = pd.to_datetime(self.df[col], errors='coerce')
                self.df[col] = self.df[col].mask(pd.isna(self.df[col]), None)

                
                missing_count = self.df[col].isna().sum()
                total_count = len(self.df)
                
                logger.info(f"Converted {col}: {original_type} -> datetime, "
                          f"{missing_count}/{total_count} missing ({missing_count/total_count*100:.1f}%)")
    
    def validate_required_columns(self):
        """Validate that required columns exist"""
        required_columns = ['po_razin_id', 'po_created_date']
        missing_columns = [col for col in required_columns if col not in self.df.columns]
        
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")
            
        logger.info("All required columns present")
    
    def initialize_calculator(self):
        """Initialize the TAT calculator"""
        logger.info(f"Initializing TAT Calculator with config: {self.config_file}")
        
        try:
            self.calculator = TATCalculator(self.config_file)
            logger.info("TAT Calculator initialized successfully")
            
        except ImportError:
            logger.error("TATCalculator not available. Please ensure tat_calculator.py is in the same directory.")
            raise
        except Exception as e:
            logger.error(f"Error initializing calculator: {e}")
            raise
    
    def run_calculations(self, sample_size: int = None):
        """Run TAT calculations on all or sample of POs"""
        if sample_size:
            df_to_process = self.df.head(sample_size)
            logger.info(f"Processing sample of {sample_size} POs")
        else:
            df_to_process = self.df
            logger.info(f"Processing all {len(df_to_process)} POs")
        
        self.results = []
        errors = []
        
        for index, row in df_to_process.iterrows():
            try:
                po_id = row.get('po_razin_id', f'Row_{index}')
                logger.info(f"Processing PO {index + 1}/{len(df_to_process)}: {po_id}")
                
                result = self.calculator.calculate_tat(row)
                self.results.append(result)
                
            except Exception as e:
                error_info = {
                    'index': index,
                    'po_id': row.get('po_razin_id', f'Row_{index}'),
                    'error': str(e),
                    'traceback': traceback.format_exc()
                }
                errors.append(error_info)
                logger.error(f"Error processing row {index}: {e}")
        
        logger.info(f"Completed calculations: {len(self.results)} successful, {len(errors)} errors")
        
        if errors:
            self.save_errors(errors)
        
        return self.results
    
    def save_errors(self, errors):
        """Save error details to file"""
        error_file = f"tat_errors_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(error_file, 'w') as f:
            json.dump(errors, f, indent=2)
        logger.info(f"Error details saved to: {error_file}")
    
    def save_results(self, filename_prefix: str = "tat_results"):
        """Save calculation results to JSON file"""
        if not self.results:
            logger.warning("No results to save")
            return
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{filename_prefix}_{timestamp}.json"
        
        with open(filename, 'w') as f:
            json.dump(self.results, f, indent=2, default=str)
        
        logger.info(f"Results saved to: {filename}")
        return filename
    
    def export_to_excel(self, filename_prefix: str = "tat_export"):
        """Export original data + calculated timestamps to Excel"""
        if not self.results:
            logger.warning("No results to export")
            return
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{filename_prefix}_{timestamp}.xlsx"
        
        self.calculator.export_to_excel(self.df, self.results, filename)
        return filename


def main():
    """Main execution function"""
    print("TAT Calculation System - Starting...")
    print("="*60)
    
    try:
        # Initialize runner
        runner = TATRunner()
        
        # Setup
        runner.setup()
        
        # Run calculations (start with sample for testing)
        sample_size = 5  # Process first 5 POs for testing
        print(f"\nRunning calculations on sample of {sample_size} POs...")
        results = runner.run_calculations(sample_size=sample_size)
        
        if results:
            # Save results
            results_file = runner.save_results()
            
            # Export to Excel
            excel_file = runner.export_to_excel()
            
            print(f"\nFiles Generated:")
            print(f"- Results: {results_file}")
            print(f"- Excel Export: {excel_file}")
            print(f"- Logs: tat_calculation.log")
            
        print(f"\nTo process all POs, change sample_size=None in the run_calculations() call")
        print("TAT Calculation completed successfully!")
        
    except Exception as e:
        logger.error(f"Error in main execution: {e}")
        print(f"Error: {e}")
        print("See tat_calculation.log for detailed error information")

SyntaxError: invalid syntax (837283707.py, line 90)

In [ ]:
runner = TATRunner()
runner.setup()

INFO:__main__:Setting up TAT calculation environment...
INFO:__main__:Loading Excel file: ts_big.xlsx
INFO:__main__:Loaded 5 rows and 38 columns
INFO:__main__:Converted po_created_date: datetime64[ns] -> datetime, 0/5 missing (0.0%)
INFO:__main__:Converted po_approval_date: datetime64[ns] -> datetime, 0/5 missing (0.0%)
INFO:__main__:Converted supplier_confirmation_date: datetime64[ns] -> datetime, 0/5 missing (0.0%)
INFO:__main__:Converted pi_invoice_approval_date: datetime64[ns] -> datetime, 3/5 missing (60.0%)
INFO:__main__:Converted pi_payment_date: datetime64[ns] -> datetime, 3/5 missing (60.0%)
INFO:__main__:Converted receive_first_prd_date: datetime64[ns] -> datetime, 2/5 missing (40.0%)
INFO:__main__:Converted prd_reconfirmed_date: datetime64[ns] -> datetime, 3/5 missing (60.0%)
INFO:__main__:Converted po_im_date_value: datetime64[ns] -> datetime, 1/5 missing (20.0%)
INFO:__main__:Converted po_sm_date_value: datetime64[ns] -> datetime, 2/5 missing (40.0%)
INFO:__main__:Converte

In [ ]:
y = next(runner.df.iterrows())[1]

In [ ]:
y

po_razin_id                     PO356103HKGA-0000015
po_created_date                  2023-06-12 00:00:00
po_approval_date                 2023-06-19 00:00:00
supplier_confirmation_date       2023-06-21 00:00:00
pi_invoice_approval_date         2023-06-25 00:00:00
pi_payment_date                  2023-08-23 00:00:00
receive_first_prd_date           2024-06-27 00:00:00
prd_reconfirmed_date                             NaT
po_im_date_value                 2024-02-22 00:00:00
po_sm_date_value                 2024-02-22 00:00:00
batch_created_ts                                 NaT
sm_signoff_ts                                    NaT
ci_invoice_approval_date                         NaT
ci_payment_date                                  NaT
qc_schedule_date                                 NaT
ffw_booking_ts                                   NaT
spd_ts                                           NaT
stock_pickup_date                                NaT
shipment_creation_date                        

In [ ]:
expr = runner.calculator.config.stages['10'].fallback_calculation.expression
result, formula = runner.calculator._evaluate_expression(expr, y)
print(result, formula)

ERROR:__main__:Error evaluating expression 'max(prd_reconfirmed_date, add_days(receive_first_prd_date, 7))': NaTType does not support strftime


None Calculation error: max(prd_reconfirmed_date, add_days(receive_first_prd_date, 7)) (NaTType does not support strftime)


In [ ]:
expr = "['5']"  # or "cond(1, 2, 3)"
result, formula = runner.calculator._evaluate_expression(expr, y)
print(result, formula)

None Calculation returned non-datetime: ['5'] = ['5']
